In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
df = pd.read_csv('/content/tash2.csv', index_col=0)
df = df.drop('location', axis=1)

FileNotFoundError: [Errno 2] No such file or directory: '/content/tash2.csv'

In [ ]:


from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=0)

homes = train_set.drop('price', axis=1)
homes_labels = train_set['price'].copy()
homes_num = homes.drop(['district'], axis=1)
homes_cat1 = ['district']


In [ ]:
homes_num

,rooms,size,level,max_levels
316,2,56.00,1,4
386,2,47.14,4,4
4024,2,79.00,13,17
927,3,87.65,7,11
66,4,95.00,2,4
...,...,...,...,...
5518,2,50.00,1,5
3547,4,80.00,2,5
1763,1,34.00,2,3
2804,2,55.00,4,4


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler

num_pipline = Pipeline([
    ('std_scaler', StandardScaler())
])


from sklearn.compose import ColumnTransformer
num_atributs = list(homes_num)

full_pipline = ColumnTransformer([
    ('num', num_pipline, num_atributs),
    ('cat1', OneHotEncoder(), homes_cat1)
])
homes_pripaired = full_pipline.fit_transform(homes)

In [ ]:
homes_pripaired

array([[-0.58340923, -0.51012034, -1.20344694, ...,  0.        ,
         0.        ,  0.        ],
       [-0.58340923, -0.80580132,  0.12582061, ...,  0.        ,
         1.        ,  0.        ],
       [-0.58340923,  0.2574488 ,  4.11362324, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [-1.55606381, -1.2443169 , -0.76035776, ...,  0.        ,
         0.        ,  0.        ],
       [-0.58340923, -0.54349291,  0.12582061, ...,  0.        ,
         0.        ,  0.        ],
       [-0.58340923,  0.85815508,  1.01199897, ...,  0.        ,
         0.        ,  1.        ]])

In [ ]:
from sklearn.linear_model import LinearRegression
LR_model = LinearRegression()

In [ ]:
LR_model.fit(homes_pripaired, homes_labels)

LinearRegression()

In [ ]:
test_data = homes.sample(10)

In [ ]:
labels = homes_labels.loc[test_data.index]
labels

6254    50000
7559    23000
4242    38000
1309    99000
1054    47500
5170    35000
4318    41744
3170    63000
3076    45000
931     36000
Name: price, dtype: int64

In [ ]:
test_data_prepared = full_pipline.transform(test_data)
predicteed_data = LR_model.predict(test_data_prepared)
predicteed_data

array([48241.14946875, 19891.60507531, 48707.86086859, 90503.78431589,
       59639.80200526, 34987.17637073, 57200.75049338, 62851.68626895,
       57463.31592317, 42929.07672359])

In [ ]:
pd.DataFrame({'labels': labels, 'predected_labels': predicteed_data})

,labels,predected_labels
6254,50000,48241.149469
7559,23000,19891.605075
4242,38000,48707.860869
1309,99000,90503.784316
1054,47500,59639.802005
5170,35000,34987.176371
4318,41744,57200.750493
3170,63000,62851.686269
3076,45000,57463.315923
931,36000,42929.076724


In [ ]:
x_test = test_set.drop('price', axis=1)
x_test

,district,rooms,size,level,max_levels
3917,Мирабадский,1,55.0,2,9
4909,Мирзо-Улугбекский,4,71.0,4,4
1080,Шайхантахурский,2,75.0,4,7
2507,Олмазорский,4,80.0,1,4
6910,Чиланзарский,3,70.0,5,5
...,...,...,...,...,...
4079,Яккасарайский,1,43.0,5,16
2575,Сергелийский,1,44.9,9,9
1679,Мирабадский,3,100.0,5,9
2543,Юнусабадский,3,112.0,2,9


In [ ]:
y_test = test_set['price'].copy()
y_test

3917     61000
4909     50000
1080     47000
2507     53500
6910     40000
         ...  
4079     37000
2575     19335
1679    100000
2543    130000
6180     63000
Name: price, Length: 1326, dtype: int64

In [ ]:
predected_data = full_pipline.transform(x_test)
predected_data

array([[-1.55606381, -0.54349291, -0.76035776, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.36189994, -0.00953177,  0.12582061, ...,  0.        ,
         0.        ,  0.        ],
       [-0.58340923,  0.12395851,  0.12582061, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.38924535,  0.95827279,  0.56890979, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.38924535,  1.35874364, -0.76035776, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.36189994,  0.69129222, -0.31726857, ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
y_predected = LR_model.predict(predected_data)
y_predected

array([ 67337.68573322,  47679.65200002,  63065.23955915, ...,
        98628.79321451, 102518.07974495,  72360.01915747])

In [ ]:
from sklearn.metrics import mean_absolute_error
mean = mean_absolute_error(y_test, y_predected)
print('mae=', mean)


mae= 12736.003200561283


In [ ]:
from sklearn.metrics import mean_squared_error
mese = mean_squared_error(y_test, y_predected)
print('mese', np.sqrt(mese))

mese 20637.141947122353


In [ ]:
from sklearn.ensemble import RandomForestRegressor
RF_model = RandomForestRegressor()
RF_model.fit(homes_pripaired, homes_labels)

RandomForestRegressor()

In [ ]:
RF = RF_model.predict(predected_data)

In [ ]:
from sklearn.metrics import mean_squared_error
mese = mean_squared_error(y_test, RF)
print('mese', np.sqrt(mese))

mese 17938.20090515873


In [ ]:
x = df.drop('price', axis=1)
y = df['price'].copy()

In [ ]:
x_prepired = full_pipline.transform(x)

In [ ]:
from sklearn.model_selection import cross_val_score
mae = cross_val_score(LR_model, x_prepired, y, scoring='neg_mean_squared_error', cv=5)
mae

array([-2.59287951e+08, -5.96763101e+08, -5.08797192e+08, -3.84691802e+08,
       -3.36351989e+08])

In [ ]:
def display_score(scores):
  print('scores', scores)
  print('Mean', scores.mean())
  print('std.dev', scores.std())

In [ ]:
display_score(np.sqrt(-mae))

scores [16102.42064451 24428.73515153 22556.53324512 19613.56168168
 18339.90154806]
Mean 20208.230454178738
std.dev 2967.4616742924695


In [ ]:
maef = cross_val_score(RF_model, x_prepired, y, scoring='neg_mean_squared_error', cv=5)
display_score(np.sqrt(-maef))

scores [15632.64982631 22115.90603738 19851.2156005  18048.14812369
 17536.56642668]
Mean 18636.89720291197
std.dev 2198.223247819296


In [ ]:
import joblib
filename = 'LR_model_Tash.jlb'
joblib.dump(LR_model, filename)


['LR_model_Tash.jlb']

In [ ]:
model = joblib.load('/content/LR_model_Tash.jlb')